In [5]:
import sys
sys.path.append('../')

from inference import Inference, get_xray_classes
import pandas as pd
import os
from glob import glob
import utils

In [ ]:
def find_predict_paths(model_dir_paths, mode):
    predict_paths = []

    for model_dir_path in model_dir_paths:
        paths = glob(model_dir_path + f'/{mode}*.csv')

        if len(paths) != 1:
            raise(Exception(f"Number of the find paths is not one! - {paths}"))

        predict_paths.append(paths[0])

    return predict_paths

def save_combined_predicts(predict_paths, save_path):
    if len(predict_paths) != 3:
        raise(Exception("Check predict_paths!"))
    
    df = None
    classes = get_xray_classes()

    for file_path in predict_paths:
        if df is None:
            df = pd.read_csv(file_path)
        else:
            df = pd.concat([df, pd.read_csv(file_path)])

    df['class_num'] = df['class'].map(classes['class2idx'])
    df = df.sort_values(by=['image_name', 'class_num'])
    df = df.drop(columns=['class_num'])

    df.to_csv(save_path, index=False)

In [7]:
prefix = '1125'
mode = 'valid'
trained_models_path = '../trained_models'
save_dir_path = 'result/single_model'

os.makedirs(save_dir_path, exist_ok=True)
finger_model_dir_paths = glob(trained_models_path + '/*/*finger*')
finger_model_dir_paths = sorted(finger_model_dir_paths)

In [8]:
for finger_model_dir_path in finger_model_dir_paths:    
    save_name = f'{prefix}_{mode}_' + os.path.basename(finger_model_dir_path).replace("finger_", "")
    save_path = os.path.join(save_dir_path, save_name + '.csv')
    save_json_path = os.path.join(save_dir_path, save_name + '.json')

    backhand_model_dir_path = finger_model_dir_path.replace('finger', 'backhand')
    arm_model_dir_path = finger_model_dir_path.replace('finger', 'arm')
    model_dir_paths = [finger_model_dir_path, backhand_model_dir_path, arm_model_dir_path]
    predict_paths = find_predict_paths(model_dir_paths, mode)
    
    dicts = {'model_dir_paths': model_dir_paths, 'predict_paths': predict_paths} # 파일 생성 정보

    save_combined_predicts(predict_paths, save_path)
    utils.save_json(dicts, save_json_path)
    

Exception: Number of the find paths is not one! - 2